In [50]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import json
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq

In [51]:
model_name = "KETI-AIR/ke-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [52]:
file_path = "C:/Users/asia/Desktop/파이널프로젝트/03_데이터전처리/순화표현모델 데이터_리라이팅완료.jsonl"

with open(file_path, "r", encoding="utf-8") as f:
    raw_data = [json.loads(line) for line in f if line.strip()]

In [53]:
def preprocess(example):
    # 1. 전체 문맥 중 마지막 줄을 따로 분리
    context_lines = example["context"][:-1]
    toxic_line = example["context"][-1]
    prefix = "rewrite last line politely"

# 2) 한 줄로 합치기 (스페이스로)
    full_input = " ".join([prefix] + context_lines + [toxic_line])

    return {
        "input_text": full_input,
        "target_text": example["output"]
    }

In [54]:
processed_data = [preprocess(example) for example in raw_data]
print(processed_data[0])

{'input_text': 'rewrite last line politely A: 오늘 지각할 뻔했다ㅋㅋ 버스 진짜 안 옴 B: 나도 거의 뛰어서 옴;; A: 수학쌤 오늘 또 지옥 시간일 듯 B: 그 ㅆㅂ 늙은이 목소리만 들어도 현타 옴', 'target_text': 'B: 수학쌤 목소리 너무 지루하긴 해'}


In [55]:
from torch.utils.data import Dataset

class RewriteDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # 1. 입력 텍스트 인코딩
        model_inputs = self.tokenizer(
            item["input_text"],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # 2. 타겟 텍스트 인코딩
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                item["target_text"],
                max_length=self.max_length,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )

        # 3. squeeze()로 batch 차원 제거
        model_inputs = {k: v.squeeze() for k, v in model_inputs.items()}
        model_inputs["labels"] = labels["input_ids"].squeeze()

        return model_inputs

In [56]:
from sklearn.model_selection import train_test_split

# 90% 학습, 10% 검증
train_data, test_data = train_test_split(processed_data, test_size=0.1, random_state=42)

# Dataset 클래스에 전달
train_dataset = RewriteDataset(train_data, tokenizer)
test_dataset = RewriteDataset(test_data, tokenizer)



In [57]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./ke-t5-rewrite-small",        # 모델 저장 위치
    evaluation_strategy="steps",               # 일정 step마다 평가
    save_strategy="steps",                     # 일정 step마다 저장
    logging_strategy="steps",                  # 로그도 일정 step마다 출력
    per_device_train_batch_size=4,             # 배치 크기 (GPU 메모리 따라 조절)
    per_device_eval_batch_size=4,
    predict_with_generate=True,                # generation을 평가에 사용
    generation_max_length=64,                  # 생성 문장 최대 길이
    generation_num_beams=4,                    # beam search 사용
    learning_rate=2e-5,                        # 기본 학습률
    num_train_epochs=3,                        # 학습 에폭 수
    weight_decay=0.01,                         # 정규화
    save_steps=500,                            # 몇 step마다 모델 저장할지
    eval_steps=500,                            # 몇 step마다 평가할지
    logging_steps=100,                         # 몇 step마다 로그 찍을지
    load_best_model_at_end=True,               # 가장 좋은 모델 자동 로딩
    metric_for_best_model="rougeL",            # 기준 평가 지표
    greater_is_better=True,                    # rougeL은 클수록 좋음
    warmup_steps=100,                          # 워밍업
    save_total_limit=2,                        # 저장할 checkpoint 수 제한
    lr_scheduler_type="linear",                # learning rate scheduler
    logging_dir="./logs",                      # 로그 파일 저장 위치
    report_to="none"                           # wandb 안 씀
)

In [58]:
import evaluate
import numpy as np

# 평가 지표 불러오기
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # -100 padding 무시하고 pad_token_id로 교체
    preds = np.where(preds == -100, tokenizer.pad_token_id, preds)
    labels = np.where(labels == -100, tokenizer.pad_token_id, labels)

    # 디코딩
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU 계산
    bleu = bleu_metric.compute(
        predictions=decoded_preds,
        references=[[label] for label in decoded_labels]
    )["bleu"]

    # ROUGE 계산
    rouge = rouge_metric.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    # 샘플 예시 5개만 확인 (선택)
    for i in range(min(5, len(decoded_preds))):
        print(f"예측: {decoded_preds[i]}")
        print(f"정답: {decoded_labels[i]}")
        print("---")

    return {
        "bleu": bleu,
        "rougeL": rouge["rougeL"]
    }

In [48]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics   # ✅ 이거만 넣으면 끝!
)

trainer.train()

C:\Users\asia\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [70]:
# 0) 깨끗한 모델 & 토크나이저 로드
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.utils.data import Dataset
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

model_name = "KETI-AIR/ke-t5-small"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 1) 아주 단순화된 preprocess
raw = raw_data[0]  # 첫 샘플
toxic_line = raw["context"][-1]
target     = raw["output"]
prefix     = "rewrite politely:"   # 맥락 없이 부드럽게 바꾸기만

sample = {
    "input_text": f"{prefix} {toxic_line}",
    "target_text": target
}

# 2) Dataset 클래스 (pad→-100 마스킹 포함)
class SimpleDataset(Dataset):
    def __init__(self, item, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.max_length= max_length
        self.item      = item

    def __len__(self): return 1

    def __getitem__(self, idx):
        inp = self.tokenizer(
            self.item["input_text"],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        lbl = self.tokenizer(
            text_target=self.item["target_text"],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )["input_ids"].squeeze()
        lbl[lbl == self.tokenizer.pad_token_id] = -100

        batch = {k: v.squeeze() for k, v in inp.items()}
        batch["labels"] = lbl
        return batch

ds = SimpleDataset(sample, tokenizer)

# 3) Overfit TrainingArguments
args = Seq2SeqTrainingArguments(
    output_dir="./ovft",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy="no",
    save_strategy="no",
    num_train_epochs=50,
    learning_rate=1e-3,
    logging_steps=5,
    predict_with_generate=False,
    load_best_model_at_end=False,
)

# 4) Trainer & train
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=ds,
    eval_dataset=ds,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model),
    compute_metrics=None
)
trainer.train()

# 5) 결과 확인
out = model.generate(
    **tokenizer(sample["input_text"], return_tensors="pt"),
    max_length=64, num_beams=1
)
print("=== Generated ===")
print(tokenizer.decode(out[0], skip_special_tokens=True))
print("=== Target ===")
print(sample["target_text"])

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,69.422500
10,28.804300
15,15.815400
20,10.593900
25,4.995900
30,4.079600
35,3.955700
40,3.787600
45,3.608800
50,3.501000


=== Generated ===
아예 진도 빠네요 하긴 과제다 늦지 예상政스럽지 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작 정작
=== Target ===
B: 수학쌤 목소리 너무 지루하긴 해
